In [23]:
from random import randint
import time
def B(eta):
    r = 0
    for i in range(eta):
        r += randint(0, 1) - randint(0, 1)
    return r


tests_list = [10, 100, 1000, 10000, 100000, 1000000]
for tests in tests_list:
    start_time = time.time()
    results = [B(2) for _ in range(tests)]
    probabilities = [results.count(i-2)/tests for i in range(5)]
    print(probabilities)
    print((time.time() - start_time)/tests)


[1/5, 3/10, 2/5, 1/10, 0]
0.00020742416381835938
[1/25, 33/100, 29/100, 13/50, 2/25]
1.0309219360351563e-05
[33/500, 29/125, 48/125, 33/125, 27/500]
1.1843204498291016e-05
[571/10000, 127/500, 927/2500, 2583/10000, 299/5000]
1.0280752182006837e-05
[6243/100000, 12577/50000, 4673/12500, 12491/50000, 6237/100000]
9.99157428741455e-06
[62867/1000000, 250421/1000000, 74917/200000, 124767/500000, 62593/1000000]
9.632575511932373e-06


In [20]:
from sage.all import parent, ZZ, vector, PolynomialRing, GF
from sage.all import log, ceil, randint, set_random_seed, random_vector, matrix, floor
import numpy as np
import random 

# Kyber parameters
n = 256
q = 3329
k = 3
eta1 = 2
eta2 = 2
Fq = GF(q)
Rq = PolynomialRing(Fq, "x")
R, x = PolynomialRing(ZZ, "x").objgen()
fx = R([1]+[0]*(n-1)+[1])

def B(eta):
    return sum(np.random.randint(0,2,eta) - np.random.randint(0,2,eta))

def string_to_bits(s):
    bits_message = []
    for letter in s:
        for i in range(6, -1, -1):
            bits_message.append(ord(letter)>>i & 1)
    return bits_message

def bits_to_string(b):
    s = []
    if len(b)==0:
        return ""
    sub = b[:7]
    i=0
    while not (sub == [0]*7) and i < len(b)//7:
        s.append(chr(int(''.join(str(item) for item in sub), 2)))
        i+=1      
        sub = b[7*i:7*i+7]   
    return "".join(s)

def map_q_to_01(polynomial):
    new_coeffs = [(coeff + q//4) % q for coeff in polynomial]
    return [int(new_coeff)*(2/q) - 0.5 for new_coeff in new_coeffs ]

def add_noise_v(v, SNR):
    noisy_v = []
    for vv in v:
        signal_power_db = 10*np.log10(q**2/4)
        noise_power_db = signal_power_db - SNR
        noise = R([round(random.normalvariate(0, 10**(noise_power_db/10))) for _ in range(0, n)])
        noisy_v.append(vv + noise)
    return noisy_v
    
def add_noise_u(u, SNR):
    noisy_u = []
    for uu in u:
        signal_power_db = 10*np.log10(q**2/4)
        noise_power_db = signal_power_db - SNR
        noise = vector(R, k, [R([ round(random.normalvariate(0, 10**(noise_power_db/10))) for _ in range(n)]) for _ in range(k)])
        noisy_u.append(uu + noise)
    return noisy_u

def generate_keys():
    A = matrix(Rq, k, k, [Rq.random_element(degree=n-1) for _ in range(k*k)])
    e = vector(R, k, [R([(B(eta1)) for _ in range(n)]) for _ in range(k)])
    s = vector(R, k, [R([(B(eta1)) for _ in range(n)]) for _ in range(k)])
    t = ( A*s + e) % fx

    return (A, t), s # (pk), sk

def encrypt(message, pk):
    A, t = pk
    if type(message) == str:
        message = string_to_bits(message)
    
    message = list(message)
    message = [message[i:i+256] for i in range(0, len(message), 256)]
    u, v = [], []
    r = vector(R, k, [R([(B(eta1)) for _ in range(n)]) for _ in range(k)])
    e_1 = vector(R, k, [R([(B(eta2)) for _ in range(n)]) for _ in range(k)])
    e_2 = R([(B(eta2)) for _ in range(n)])
    for submessage in message:
        u.append((r*A + e_1) % fx)
        v.append((r*t + e_2 + q//2 * R(submessage)) % fx)
    return u, v

def decrypt(u, v, sk, for_pc = False):
    message = []
    for i in range(0, len(v)):
        decrypted_message = (v[i] - sk*u[i]) % fx
        if for_pc:
            message.append(map_q_to_01(decrypted_message))
        else:
            message.append([int(coef > q//4 and coef < 3*q//4) for coef in decrypted_message])
    if for_pc:
        return np.array([bit for submessage in message for bit in submessage])
    else:
        all_messages = [bit for submessage in message for bit in submessage]
        return "".join(bits_to_string(bits) for bits in [message[i:i+7] for i in range(0, len(message), 7)])
    



In [22]:
import unittest
import os
from aes256_ctr_drbg import AES256_CTR_DRBG
import string
import random

    
class TestKyber(unittest.TestCase):
    probability_of_bit_flip_in_channel = 10**-9 # typical
    """
    Test Kyber levels for internal
    consistency by generating keypairs
    and shared secrets.
    """

    def generic_test_kyber(self, kyber, count):
        # Set Kyber parameters 
        n, k, q, eta1, eta2, du, dv = set_parameters(kyber)
        for _ in range(count):
            pk,sk = generate_keys()
            for _ in range(count):
                org_message = ''.join(random.SystemRandom().choice(string.ascii_uppercase + string.digits) for _ in range(10 ** _))
                message = string_to_bits(org_message)
                print(_, "Length of org text:", len(string_to_bits(message)))
                u, v = encrypt(string_to_bits(message), pk)
                noisy_v = add_noise_v(v, probability_of_bit_flip_in_channel)
                noisy_u = add_noise_u(u, probability_of_bit_flip_in_channel)
                decrypted = decrypt(noisy_u, noisy_v, sk)
                self.assertEqual(org_message, decrypted)
    
    def test_kyber512(self):
        self.generic_test_kyber("kyber512", 5)
        print("Ahihi")
        
    def test_kyber768(self):
        self.generic_test_kyber("kyber768", 5)
        
    def test_kyber1024(self):
        self.generic_test_kyber("kyber1024", 5)
        

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

EEEE

Done set parameters kyber1024


EE

Done set parameters kyber512
Done set parameters kyber768


E
ERROR: test_kyber1024_known_answer (__main__.TestKnownTestValues)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-12-3dcfd7cc1c36>", line 97, in test_kyber1024_known_answer
    return self.generic_test_kyber_known_answer(Kyber1024, "assets/PQCkemKAT_3168.rsp")
NameError: name 'Kyber1024' is not defined

ERROR: test_kyber512_known_answer (__main__.TestKnownTestValues)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-12-3dcfd7cc1c36>", line 91, in test_kyber512_known_answer
    return self.generic_test_kyber_known_answer(Kyber512, "assets/PQCkemKAT_1632.rsp")
NameError: name 'Kyber512' is not defined

ERROR: test_kyber768_known_answer (__main__.TestKnownTestValues)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-12-3dcfd7cc1c36>", line 94, in test_

In [21]:
def set_parameters(kyberX):
    if kyberX == "kyber512":
        n = 256
        k = 2
        q = 3329
        eta1 = 3
        eta2 = 2
        du = 10
        dv = 4
        print("Done set parameters kyber512")
    elif kyberX == "kyber768":
        n = 256
        k = 3
        q = 3329
        eta1 = 2
        eta2 = 2
        du = 10
        dv = 4
        print("Done set parameters kyber768")
    elif kyberX == "kyber1024": 
        n = 256
        k = 4
        q = 3329
        eta1 = 2
        eta2 = 2
        du = 11
        dv = 5
        print("Done set parameters kyber1024")
    return n, k, q, eta1, eta2, du, dv

In [7]:
!pip install pycryptodome
!pip install -r requirements.txt

  Using cached pycryptodome-3.14.1-cp35-abi3-manylinux2010_x86_64.whl (2.0 MB)
  Attempting uninstall: pycryptodome
    Found existing installation: pycryptodome 3.20.0
    Uninstalling pycryptodome-3.20.0:
      Successfully uninstalled pycryptodome-3.20.0


In [ ]:
print("Finished!")